In [2]:
import pandas as pd
from collections import Counter
import sys
import os
import ast
import json
import pandas as pd
import numpy as np
import ast
import joblib
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from xgboost import XGBClassifier
from scipy.stats import randint, uniform
import mlflow
import mlflow.sklearn
from scipy.stats import loguniform

from new_or_used import build_dataset
pd.set_option('display.max_columns', None)

mlflow.set_tracking_uri("file:../Experiments") 
mlflow.set_experiment("LogisticRegression_Experiment")


<Experiment: artifact_location=('file:///Users/sebastian/Library/Mobile '
 'Documents/com~apple~CloudDocs/Meli/CodeExercise DS - Items Nuevos_Usados '
 '(1)/MercadoLibre_Test/Notebooks/../Experiments/453329136426596343'), creation_time=1747889644004, experiment_id='453329136426596343', last_update_time=1747889644004, lifecycle_stage='active', name='LogisticRegression_Experiment', tags={}>

In [3]:
# Cargar los datasets preprocesados
X_train = pd.read_csv("../Data/Gold/X_train_gold.csv")
X_test = pd.read_csv("../Data/Gold/X_test_gold.csv")
y_train = pd.read_csv("../Data/Gold/y_train_gold.csv")
y_test = pd.read_csv("../Data/Gold/y_test_gold.csv")

y_test.rename(columns={'0': 'condition'}, inplace=True)

y_train_final = y_train["condition"].map({"used": 0, "new": 1})
y_test_final = y_test["condition"].map({"used": 0, "new": 1})


print("Datasets cargados correctamente.")

Datasets cargados correctamente.


In [4]:
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score
from scipy.stats import loguniform

# === Agrupar top ciudades
top_cities = X_train['seller_address.city.name'].value_counts().nlargest(20).index
X_train['city_grouped'] = X_train['seller_address.city.name'].apply(lambda x: x if x in top_cities else 'other')
X_test['city_grouped'] = X_test['seller_address.city.name'].apply(lambda x: x if x in top_cities else 'other')

# === Eliminar columnas de alta cardinalidad
X_train_lr = X_train.drop(columns=['seller_id', 'seller_address.city.name'])
X_test_lr = X_test.drop(columns=['seller_id', 'seller_address.city.name'])

# === Submuestreo para evitar explosión de RAM
X_sample = X_train_lr.sample(frac=0.3, random_state=42)
y_sample = y_train_final.loc[X_sample.index]

# === Columnas
cat_cols = X_sample.select_dtypes(include='object').columns.tolist()
num_cols = X_sample.select_dtypes(include=['int64', 'float64']).columns.tolist()

# === Preprocesamiento
preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols),
    ("num", StandardScaler(), num_cols)
])

# === Pipeline base
pipeline = Pipeline([
    ("preprocessing", preprocessor),
    ("classifier", LogisticRegression(max_iter=1000, random_state=42))
])

# === Búsqueda de hiperparámetros
param_dist = {
    "classifier__C": loguniform(0.01, 10),
    "classifier__solver": ["lbfgs", "saga"]
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter=5,
    scoring="accuracy",
    cv=cv,
    random_state=42,
    verbose=2,
    n_jobs=-1
)

# === MLflow
mlflow.set_tracking_uri("file:../Experiments")
mlflow.set_experiment("LogisticRegression_Experiment")

with mlflow.start_run(run_name="LogReg_CV_Sample"):
    search.fit(X_sample, y_sample)

    best_pipeline = search.best_estimator_
    mlflow.log_params(search.best_params_)

    # Evaluar en full test
    y_pred = best_pipeline.predict(X_test_lr)
    y_prob = best_pipeline.predict_proba(X_test_lr)[:, 1]

    acc = accuracy_score(y_test_final, y_pred)
    auc = roc_auc_score(y_test_final, y_prob)

    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("roc_auc", auc)
    mlflow.sklearn.log_model(best_pipeline, "LogisticRegression_CV")

    print(f"✅ Logistic Regression Accuracy: {acc:.4f} | ROC AUC: {auc:.4f}")

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] END classifier__C=0.13292918943162169, classifier__solver=lbfgs; total time= 5.8min
[CV] END classifier__C=0.13292918943162169, classifier__solver=lbfgs; total time= 6.1min
[CV] END classifier__C=0.13292918943162169, classifier__solver=lbfgs; total time= 6.4min
[CV] END classifier__C=0.13292918943162169, classifier__solver=lbfgs; total time= 6.4min
[CV] END classifier__C=0.13292918943162169, classifier__solver=lbfgs; total time= 6.5min
[CV] END classifier__C=0.6251373574521749, classifier__solver=lbfgs; total time=11.7min
[CV] END classifier__C=0.21751953118777648, classifier__solver=lbfgs; total time= 6.6min
[CV] END classifier__C=0.6251373574521749, classifier__solver=lbfgs; total time= 6.8min
[CV] END classifier__C=0.6251373574521749, classifier__solver=lbfgs; total time= 7.4min
[CV] END classifier__C=0.6251373574521749, classifier__solver=lbfgs; total time= 8.6min
[CV] END classifier__C=0.21751953118777648, classifier

2025/05/22 12:28:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Logistic Regression Accuracy: 0.8467 | ROC AUC: 0.9207
